In [24]:
# Import packages

import math
import json
import re
import bs4
import csv
import requests

In [67]:
base_url = 'https://nextrequest.cabq.gov/requests'
req = requests.get(base_url)

for i in range(2, 479):
    html = req.text.encode('utf8')
    soup = bs4.BeautifulSoup(html, "html5lib")
    rl = soup.tbody.find_all('tr', class_ = " demo-data-false") 
    for prr in rl:
        cl = list(prr.children)
        # get information from main page
        req_id = cl[1].a.strong.contents[0]

print(req_id)

18-3673


In [2]:
base_url = 'https://nextrequest.cabq.gov/requests'
url_det = base_url + '/' + '18-3645'

In [10]:
url_det

'https://nextrequest.cabq.gov/requests/18-3645'

In [52]:
req_det = requests.get('https://bainbridgewa.nextrequest.com/requests/18-21')
html = req_det.text.encode('utf8')
soup = bs4.BeautifulSoup(html, "html5lib")
rdl = soup.find_all(id = 'request-text')

In [53]:
print(rdl)

[<div class="small-12 columns" id="request-text">
      <p>BLD# 7541-1999. 10253 NE Darden Lane, Bainbridge Island, WA.  Single family residence house plans BLD #7541-1999.</p>
    </div>]


In [56]:
time= soup.find_all('span', class_ = 'time-quotes')

In [57]:
time

[<span class="time-quotes">
   January 22, 2018,  4:03pm
 </span>, <span class="time-quotes">
   January 17, 2018,  9:40am
 </span>, <span class="time-quotes">
   January 17, 2018,  9:38am
 </span>, <span class="time-quotes">
   January 12, 2018,  5:55pm
 </span>, <span class="time-quotes">
   January  5, 2018, 12:10pm
 </span>, <span class="time-quotes">
   January  5, 2018, 12:10pm
 </span>]

In [58]:
close_date = time[0].get_text().lstrip().rstrip()

In [59]:
close_date

'January 22, 2018,  4:03pm'

In [47]:
req_det = requests.get('https://nextrequest.cabq.gov/requests/18-554')
html = req_det.text.encode('utf8')
soup = bs4.BeautifulSoup(html, "html5lib")
rdl = soup.find_all(id = 'request-text')
print(rdl)

[]


In [66]:
req_det = requests.get('https://cityoflascruces.nextrequest.com/requests/18-135')
html = req_det.text.encode('utf8')
soup = bs4.BeautifulSoup(html, "html5lib")
rdl = soup.find_all(id = 'request-text')
print(rdl)
try:
    #request = rdl[0].p.contents[0].lstrip().rstrip()
    request = rdl[0].get_text().rstrip().lstrip()
except:
    request = 'error'
    print(req_id)
rd = soup.find_all('p', class_= "request_date") 
print(rd)
try:
    create_date = re.findall(r'([^\n]+)', rd[0].get_text().lstrip().rstrip())[0]
except:
    create_date = 'error'
    print(req_id)
time= soup.find_all('span', class_ = 'time-quotes')
if status == 'closed':
    close_date = time[0].get_text().lstrip().rstrip()
else:
    close_date = 'NaN'
    
print(close_date)

[<div class="small-12 columns" id="request-text">
      <p>Please let us know the property has any open Code Violations &amp; Permits[open/expired] ?</p>
<p> </p>
<p>Property Address: 3507 SIERRA BONITA AVENUE, LAS CRUCES NM           88012</p>
<p> </p>
<p>Parcel#: 4009130111136</p>
<p> </p>
<p>If any inspection need before closing?</p>
    </div>]
[<p class="request_date">
          <strong>March  6, 2018</strong>
          via email
        </p>]
March  9, 2018, 10:01am


In [13]:
time = soup.find_all('div', class_ = 'request-history')

In [6]:
create_date = re.findall(r'([^\n]+)', rdl[0].get_text().lstrip().rstrip())[0]

In [18]:
time= soup.find_all('span', class_ = 'time-quotes')

In [21]:
time[0].get_text().lstrip().rstrip()

'June 14, 2018, 11:12am'

In [64]:
if 'Request Published' in rl[0].get_text() or 'Request Closed' in rl[0].get_text():
    close_date = rl[0].find_all('span', class_ = 'time-quotes')[0].get_text().lstrip().rstrip()
    print(close_date)

June 28, 2018,  2:04pm


In [28]:
# Scrape data from Albuquerque Next Request portal which has different url/data format than others

cities = ['Albuquerque']

for city in cities:
    fn = '{}3.csv'.format(city)
    with open(fn, 'w') as csvfile:
        fieldnames = ['Reference No', 'Request Status', 'requester', 'request', 'departments', 'cost', 'PoC', 'Create Date', 'Close Date']
        writer = csv.DictWriter(csvfile, fieldnames = fieldnames, delimiter = ',')

        writer.writeheader()
        base_url = 'https://nextrequest.cabq.gov/requests'
        alb2_url = 'https://nextrequest.cabq.gov/requests?requests_smart_listing[page]=260'
        req = requests.get(alb2_url)

        for i in range(260, 261):
            html = req.text.encode('utf8')
            soup = bs4.BeautifulSoup(html, "html5lib")
            rl = soup.tbody.find_all('tr', class_ = " demo-data-false") 
            for prr in rl:
                cl = list(prr.children)
                # get information from main page
                req_id = cl[1].a.strong.contents[0]
                status = cl[3]['class'][1]
                requester = cl[5].get_text().lstrip().rstrip()
                departments = cl[9].get_text().lstrip().rstrip()
                cost = cl[11].get_text().lstrip().rstrip().strip('$')
                poc = cl[13].get_text().lstrip().rstrip()
                
                #get information from request detail page
                url_det = base_url + "/" + req_id
                req_det = requests.get(url_det)
                html = req_det.text.encode('utf8')
                soup = bs4.BeautifulSoup(html, "html5lib")
                rdl = soup.find_all(id = 'request-text')
                try:
                    #request = rdl[0].p.contents[0].lstrip().rstrip()
                    request = rdl[0].get_text().rstrip().lstrip()
                except:
                    request = 'error'
                    print(req_id)
                rd = soup.find_all('p', class_= "request_date") 
                create_date = re.findall(r'([^\n]+)', rd[0].get_text().lstrip().rstrip())[0]
                time= soup.find_all('span', class_ = 'time-quotes')
                if status == 'closed':
                    close_date = time[0].get_text().lstrip().rstrip()
                else:
                    close_date = 'NaN'
                
                #write to csv
                writer.writerow({'Reference No': req_id,
                                'Request Status': status,
                                'requester': requester,
                                'request': request,
                                'departments': departments,
                                'cost': cost,
                                'PoC': poc,
                                'Create Date': create_date,
                                'Close Date': close_date})
                
            url = 'https://nextrequest.cabq.gov/requests?requests_smart_listing[page]={}'.format(i)
            req = requests.get(url)

In [75]:
# Scrape data from Next Request portals from list of cities given below
#'bainbridgewa', 'cityoflascruces', 'mercerisland', 'miami','middleboroughma', 
cities = ['nola', 'oaklandca', 'providenceri', 'sanfrancisco', 'vallejo', 'westsacramento' ]

for city in cities:
    print(city)
    fn = '{}4.csv'.format(city)
    with open(fn, 'w') as csvfile:
        fieldnames = ['Reference No', 'Request Status', 'requester', 'request', 'departments', 'PoC', 'Create Date', 'Close Date']
        writer = csv.DictWriter(csvfile, fieldnames = fieldnames, delimiter = ',')

        writer.writeheader()
        url = 'https://{}.nextrequest.com/requests'.format(city)
        print(url)
        
        req = requests.get(url)
        html = req.text.encode('utf8')
        soup = bs4.BeautifulSoup(html, "html5lib")
        
        num_rec = int(soup.find_all('h2')[1].span.get_text().lstrip().rstrip())
        print(num_rec)
        num_page = math.ceil(num_rec/25)
        print(num_page)

        for i in range(126, num_page + 2):
            rl = soup.tbody.find_all('tr', class_ = " demo-data-false") 
            for prr in rl:
                cl = list(prr.children)
                # get information from main page
                req_id = cl[1].a.strong.contents[0]
                status = cl[3]['class'][1]
                requester = cl[5].get_text().lstrip().rstrip()
                departments = cl[9].get_text().lstrip().rstrip()
                poc = cl[11].get_text().lstrip().rstrip()        
                
            #get information from request detail page
                url_det = url + "/" + req_id
                req_det = requests.get(url_det)
                html = req_det.text.encode('utf8')
                soup = bs4.BeautifulSoup(html, "html5lib")
                rdl = soup.find_all(id = 'request-text')
                try:
                    rdlz = rdl[0]
                except:
                    print(url_det)
                try:
                    #request = rdl[0].p.contents[0].lstrip().rstrip()
                    request = rdlz.get_text().rstrip().lstrip()
                except:
                    request = 'error'
                    print(url_det)
                rd = soup.find_all('p', class_= "request_date") 
                rdz = rd[0]
                try:
                    create_date = re.findall(r'([^\n]+)', rdz.get_text().lstrip().rstrip())[0]
                except:
                    create_date = 'error'
                    print(url_det)
                time= soup.find_all('span', class_ = 'time-quotes')
                if status == 'closed':
                    tz = time[0]
                    try:
                        close_date = tz.get_text().lstrip().rstrip()
                    except:
                        close_date = 'NaN'
                        print(url_det)
                else:
                    close_date = 'NaN'
                
                #write to csv
                writer.writerow({'Reference No': req_id,
                                'Request Status': status,
                                'requester': requester,
                                'request': request,
                                'departments': departments,
                                'PoC': poc,
                                'Create Date': create_date,
                                'Close Date': close_date})
                    
            url_2 = url+'?requests_smart_listing[page]={}'.format(i)
            req = requests.get(url_2)
            html = req.text.encode('utf8')
            soup = bs4.BeautifulSoup(html, "html5lib")

nola
https://nola.nextrequest.com/requests
3428
138
oaklandca
https://oaklandca.nextrequest.com/requests
27375
1095


KeyboardInterrupt: 